In [1]:
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np

Using TensorFlow backend.


In [2]:
def load_image():
    r = []
    g = []
    b = []
    for i in range(0, 8000):
        if i + 1 < 10:
            location = '000' + str(i+1)
        elif i + 1 < 100:
            location = '00' + str(i+1)
        elif i + 1 < 1000:
            location = '0' + str(i+1)
        else:
            location = str(i+1)
        test_image = image.load_img('./solved_data/0' + location + '.jpg', target_size = (80, 80))
        test_image = image.img_to_array(test_image)
        tmp_r = []
        tmp_g = []
        tmp_b = []
        
        for j in range(0, 80):
            tmp = []
            for k in range(0, 80):
                tmp.append(test_image[j][k][0])
            tmp_r.append(tmp)
        r.append(tmp_r)

        for j in range(0, 80):
            tmp = []
            for k in range(0, 80):
                tmp.append(test_image[j][k][1])
            tmp_g.append(tmp)
        g.append(tmp_g)
        
        for j in range(0, 80):
            tmp = []
            for k in range(0, 80):
                tmp.append(test_image[j][k][2])
            tmp_b.append(tmp)
        b.append(tmp_b)
    return r, g, b
data_r, data_g, data_b = load_image()

In [3]:
pca = PCA(n_components = 10, copy = False)
solved_train_r = []
solved_train_g = []
solved_train_b = []
for i in range(0, 6000):
    tmp = np.array(data_r[i])
    tmp = pca.fit_transform(tmp)
    tmp = tmp.reshape(800, 1)
    solved_train_r.append(tmp)
    
    tmp = np.array(data_g[i])
    tmp = pca.fit_transform(tmp)
    tmp = tmp.reshape(800, 1)
    solved_train_g.append(tmp)
    
    tmp = np.array(data_b[i])
    tmp = pca.fit_transform(tmp)
    tmp = tmp.reshape(800, 1)
    solved_train_b.append(tmp)

In [4]:
train_r = np.array(solved_train_r)
train_g = np.array(solved_train_g)
train_b = np.array(solved_train_b)

In [5]:
train_r = train_r.reshape(6000, 800)
train_g = train_g.reshape(6000, 800)
train_b = train_b.reshape(6000, 800)

In [6]:
solved_test_r = []
solved_test_g = []
solved_test_b = []
for i in range(6000, 8000):
    tmp = np.array(data_r[i])
    tmp = pca.fit_transform(tmp)
    tmp = tmp.reshape(800, 1)
    solved_test_r.append(tmp)
    
    tmp = np.array(data_g[i])
    tmp = pca.fit_transform(tmp)
    tmp = tmp.reshape(800, 1)
    solved_test_g.append(tmp)
    
    tmp = np.array(data_b[i])
    tmp = pca.fit_transform(tmp)
    tmp = tmp.reshape(800, 1)
    solved_test_b.append(tmp)

In [7]:
test_r = np.array(solved_test_r)
test_g = np.array(solved_test_g)
test_b = np.array(solved_test_b)

In [8]:
test_r = test_r.reshape(2000, 800)
test_g = test_g.reshape(2000, 800)
test_b = test_b.reshape(2000, 800)

In [9]:
import pandas as pd
train_dataset = pd.read_csv('annotations.csv')
target = train_dataset['class']

In [10]:
y_train = []
for i in range(0, 6000):
    y_train.append(int(target[i]))
y_test = []
for i in range(6000, 8000):
    y_test.append(int(target[i]))

In [11]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [12]:
target_train  = np_utils.to_categorical(y_train, 197)
target_test  = np_utils.to_categorical(y_test, 197)

In [13]:
def neural_network(solved_train_r, solved_train_g, solved_train_b, solved_test_r, solved_test_g, solved_test_b, 
                   train_r, train_g, train_b, test_r, test_g, test_b, train_target, test_target):
    model1=Sequential()
    model1.add(Dense(input_dim = 800 ,units = 600,activation='relu'))
    model1.add(Dropout(0.3))
    model1.add(Dense(units = 400,activation = 'relu'))
    model1.add(Dropout(0.1))
    model1.add(Dense(units = 197,activation = 'softmax'))
    model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model1.fit(train_r, train_target, batch_size = 100, epochs = 10, validation_split = 0.1) #batch_size越大越利用GPU平行计算，速度越快
    score = model1.evaluate(test_r, test_target)
    print("test_r accuracy:{}%".format(int(score[1] * 10000)/100))
    result_train_r = []
    for i in range(0, 6000):
        tmp = np.array(solved_train_r[i])
        tmp = tmp.reshape(1,800)
        tmp = model1.predict_classes(tmp)
        result_train_r.append(int(tmp))
    result_test_r = []
    for i in range(0, 2000):
        tmp = np.array(solved_test_r[i])
        tmp = tmp.reshape(1,800)
        tmp = model1.predict_classes(tmp)
        result_test_r.append(int(tmp))
    
    model2=Sequential()
    model2.add(Dense(input_dim = 800 ,units = 600,activation='relu'))
    model2.add(Dropout(0.3))
    model2.add(Dense(units = 400,activation = 'relu'))
    model2.add(Dropout(0.1))
    model2.add(Dense(units = 197,activation = 'softmax'))
    model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model2.fit(train_g, train_target, batch_size = 100, epochs = 10, validation_split = 0.1) #batch_size越大越利用GPU平行计算，速度越快
    score = model2.evaluate(test_g, test_target)
    print("test_g accuracy:{}%".format(int(score[1] * 10000)/100))
    result_train_g = []
    for i in range(0, 6000):
        tmp = np.array(solved_train_g[i])
        tmp = tmp.reshape(1,800)
        tmp = model2.predict_classes(tmp)
        result_train_g.append(int(tmp))
    result_test_g = []
    for i in range(0, 2000):
        tmp = np.array(solved_test_g[i])
        tmp = tmp.reshape(1,800)
        tmp = model2.predict_classes(tmp)
        result_test_g.append(int(tmp))
        
    model3=Sequential()
    model3.add(Dense(input_dim = 800 ,units = 600,activation='relu'))
    model3.add(Dropout(0.3))
    model3.add(Dense(units = 400,activation = 'relu'))
    model3.add(Dropout(0.1))
    model3.add(Dense(units = 197,activation = 'softmax'))
    model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model3.fit(train_b, train_target, batch_size = 100, epochs = 10, validation_split = 0.1) #batch_size越大越利用GPU平行计算，速度越快
    score = model3.evaluate(test_b, test_target)
    print("test_b accuracy:{}%".format(int(score[1] * 10000)/100))
    result_train_b = []
    for i in range(0, 6000):
        tmp = np.array(solved_train_b[i])
        tmp = tmp.reshape(1,800)
        tmp = model3.predict_classes(tmp)
        result_train_b.append(int(tmp))
    result_test_b = []
    for i in range(0, 2000):
        tmp = np.array(solved_test_b[i])
        tmp = tmp.reshape(1,800)
        tmp = model3.predict_classes(tmp)
        result_test_b.append(int(tmp))
    
    result_train = []
    for i in range(0, 6000):
        tmp = []
        tmp.append(result_train_r[i])
        tmp.append(result_train_g[i])
        tmp.append(result_train_b[i])
        result_train.append(tmp)
    result_train = np.array(result_train)
    result_train = result_train.reshape(6000, 3)
    result_test = []
    for i in range(0, 2000):
        tmp = []
        tmp.append(result_test_r[i])
        tmp.append(result_test_g[i])
        tmp.append(result_test_b[i])
        result_test.append(tmp)
    result_test = np.array(result_test)
    result_test = result_test.reshape(2000, 3)
    
    model4=Sequential()
    model4.add(Dense(input_dim=3,units=300,activation='relu'))
    model4.add(Dropout(0.1))
    model4.add(Dense(units=197,activation='softmax'))
    model4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model4.fit(result_train, target_train, batch_size=50, epochs=10) #batch_size越大越利用GPU平行计算，速度越快
    score = model4.evaluate(result_test, test_target)
    print("final_test accuracy:{}%".format(int(score[1] * 10000)/100))

neural_network(solved_train_r, solved_train_g, solved_train_b, solved_test_r, solved_test_g, solved_test_b, 
               train_r, train_g, train_b, test_r, test_g, test_b, target_train, target_test)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 5400 samples, validate on 600 samples
Epoch 1/10
5400/5400 [==============================] - 2s 327us/step - loss: 16.0276 - acc: 0.0054 - val_loss: 16.0106 - val_acc: 0.0067
Epoch 2/10
5400/5400 [==============================] - 0s 61us/step - loss: 16.0167 - acc: 0.0063 - val_loss: 15.9569 - val_acc: 0.0100
Epoch 3/10
5400/5400 [==============================] - 0s 61us/step - loss: 16.0375 - acc: 0.0050 - val_loss: 15.9569 - val_acc: 0.0100
Epoch 4/10
5400/5400 [==============================] - 0s 60us/step - loss: 16.0375 - acc: 0.0050 - val_loss: 15.9569 - val_acc: 0.0100
Epoch 5/10
5400/5400 [==============================] - 0s 61us/step - loss: 16.0345 - acc: 0.0052 - val_loss: 15.9569 - val_acc: 0.0100
Epoch 6/10
5400/5400 [==========